In [10]:
from pandas.tseries.offsets import DateOffset
import ee
import yaml
import time
import os
import json
import pandas as pd
from utils import exportImageToGDrive,exportImageToGCS
from download_sen12 import *
import subprocess

ee.Initialize()

In [11]:
config_dict = load_config('config.yml')
source = config_dict['data_list'][0]
sensor = config_dict['sensors'][1]
# export_folder = config_dict['drive_folder']
export_folder = config_dict['bucket']

print(export_folder)

project-canopy-temp-2


In [12]:
sensor['bands']

['B2', 'B3', 'B4', 'B8', 'B8A', 'TCI_R', 'TCI_G', 'TCI_B', 'NDVI']

In [13]:
source

{'name': ['full_basin_export_v2'],
 'start_date': '2019-01-01',
 'end_date': '2020-12-31',
 'geometry': 'point',
 'size': 20000,
 'resolution': 10,
 'sort_by': 'name',
 'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI',
 'sensors': [0, 1, 2]}

In [14]:
start = pd.to_datetime('01-01-2020')

start

Timestamp('2020-01-01 00:00:00')

In [15]:
def import_aois(csv_loc, Full_Congo_Pull=False, start_date=None,
                end_date=None, days_duration=90, poly_start=0, poly_limit=None):
    features = []
    polygons = []
    day_offset = days_duration / 2
    start_end_list = []
    
    if Full_Congo_Pull:
        with open(csv_loc,"r",encoding='utf-8') as jsonfile:
            data = json.load(jsonfile)
            for geometry in data["features"]:
                polygon = geometry["geometry"]["coordinates"][0][0]
                poly_obj = ee.Geometry.Polygon(polygon)
                feature = ee.Feature(poly_obj)
                features.append(feature)
                
            original_date = pd.to_datetime('01-01-2020')
            start = (original_date + DateOffset(days=-30))
            end = (original_date + DateOffset(days=30))
            start_date = str(start)[:10]
            end_date = str(end)[:10]
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': 30,
                'area': 1
            }
            
            start_end_list = [date_dict] * len(features)
            
            return ee.FeatureCollection(features), start_end_list
            
    else:
        feature_id = poly_start
        
        if poly_limit:
            df_labels = pd.read_csv(csv_loc, skiprows=range(1, poly_start+1), nrows=poly_limit)
        else:
            df_labels = pd.read_csv(csv_loc, skiprows=range(1, poly_start+1))

        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined","tile date","area (km2)"]]
        df_labels["tile date"] = pd.to_datetime(df_labels["tile date"])
        start = (df_labels["tile date"] + DateOffset(days=-day_offset))
        end = (df_labels["tile date"] + DateOffset(days=day_offset))
        for i in range(len(start)):
            start_date = str(start[i])[:10]
            end_date = str(end[i])[:10]
            original_date = df_labels.loc[i, 'tile date']
            area = df_labels.loc[i, 'area (km2)']
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': day_offset,
                'area': area
            }
            start_end_list.append(date_dict)

        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])
            
        for poly in polygons:
            # create an roi. first item in Misha's label list
            feature_id += 1 
            # create geometry object, create feature object, append to features list for feature collection creation 
            polys = ee.Geometry.Polygon(poly)
            feature = ee.Feature(polys,{"name":feature_id})
            features.append(feature)
            
        return ee.FeatureCollection(features),start_end_list,df_labels


# ### CHANGE BELOW PATH ###
# zhenya_label_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv"
# david__label_path = 'D:/canopy_data/csvs/polygons_101320.csv'
# zhenya_tilelist_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/shub_tile_list/tiles_polygons_full.csv"
# david_grid_path = 'D:/canopy_data/Geometry/Congo_Basin_Shapefile_Grid_v1.geojson'
# zhenya_grid_path = '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/polygon_shapefiles/Congo_Basin_Shapefile_Grid_v1.geojson'
zhenya_grid_path_2 = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/congo_basin_boundary/Congo_Basin_Boundary_no_islands_v5_gridded.geojson"

In [16]:
fc,date_range_list = import_aois(zhenya_grid_path_2, Full_Congo_Pull=True)

In [17]:
# date_range_list[0:5]

In [18]:
offset_dict = {
    30: 'two years'
}

In [ ]:
exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc,offset_dict,date_range_list,loop_start=545,debug=True
)

2403 features have been loaded


# Get Status on Exports

In [ ]:
folder_ouput_match = "full_basin_export_v2"

def gee_download_status(folder_output_match):
    d = {}
    output = subprocess.run(['earthengine','task','list'], stdout=subprocess.PIPE)
    results = output.stdout.decode("utf-8").split("\n")[2:-1]
    results_filtered = [i for i in results if "_".join(i.split(" ")[4].split("_")[1:5]) == folder_output_match]

    
    tile_ids = [i.split("_")[0].split("  ")[-1] for i in results_filtered]
    statuses = [i.split("  ")[3] for i in results_filtered]
    
    for index,tile_id in enumerate(tile_ids):
        d[tile_id] = statuses[index] 
        
    completed = sum(x == "COMPLETED" for x in d.values()) 
    
    print(f'{completed} of {len(results_filtered)} have been downloaded for the {folder_output_match} pull')
    
    return results
    

In [ ]:
results = gee_download_status(folder_ouput_match)

In [160]:
folder_ouput_match = "full_basin_export_v2"
results_filtered,d = gee_download_status(folder_ouput_match)

545 of 2403 have been downloaded for the full_basin_export_v2 pull


In [162]:
d["1"]

'COMPLETED'

In [158]:
!earthengine task info OR2JDLEGL2OINFR2BHEGRKGT

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

OR2JDLEGL2OINFR2BHEGRKGT:
  State: READY
  Type: Export.image
  Description: 2403_full_basin_export_v2_Dec_13_19:57:59_2020
  Created: 2020-12-13 19:58:05.318000
  Started: 1969-12-31 19:00:00
  Updated: 2020-12-13 19:58:05.318000


In [156]:
print(*results_filtered, sep='\n')

OR2JDLEGL2OINFR2BHEGRKGT  Export.image  2403_full_basin_export_v2_Dec_13_19:57:5..  READY      ---
5RLLUDNQGAEA2L65WTZHCHBV  Export.image  2402_full_basin_export_v2_Dec_13_19:57:3..  READY      ---
YWQRNJ76KFM2ZGH4ZMP56U6M  Export.image  2401_full_basin_export_v2_Dec_13_19:57:0..  READY      ---
LZ4IQOQM2KJYAPMZ3EBSTJUY  Export.image  2400_full_basin_export_v2_Dec_13_19:56:5..  READY      ---
5WI5YKDYHKB7KAE3KHASHR6U  Export.image  2399_full_basin_export_v2_Dec_13_19:56:4..  READY      ---
LXEYGOZR3YB33W5OJSBKRHM4  Export.image  2398_full_basin_export_v2_Dec_13_19:56:2..  READY      ---
MJR3DFVIG6BRTQBWTD3LKULE  Export.image  2397_full_basin_export_v2_Dec_13_19:55:5..  READY      ---
GZLTR35SARQUKOBNEZSU4KQE  Export.image  2396_full_basin_export_v2_Dec_13_19:55:1..  READY      ---
SDYWC42ISKVXICP7MSUWRKA5  Export.image  2395_full_basin_export_v2_Dec_13_19:54:3..  READY      ---
EQV7QIIBUKGB2XWF37I23AJM  Export.image  2394_full_basin_export_v2_Dec_13_19:53:2..  READY      ---
ET6LI7VTKH

In [7]:
## testing try/except loop

exports, exceptions = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, date_range_list
)

5047 features have been loaded
processing polygon 1 of 5047
processing polygon 2 of 5047
processing polygon 3 of 5047
processing polygon 4 of 5047
processing polygon 5 of 5047
processing polygon 6 of 5047
processing polygon 7 of 5047
processing polygon 8 of 5047
processing polygon 9 of 5047
processing polygon 10 of 5047
processing polygon 11 of 5047
processing polygon 12 of 5047
processing polygon 13 of 5047
processing polygon 14 of 5047
processing polygon 15 of 5047
processing polygon 16 of 5047
processing polygon 17 of 5047
processing polygon 18 of 5047
processing polygon 19 of 5047
processing polygon 20 of 5047
processing polygon 21 of 5047
processing polygon 22 of 5047
processing polygon 23 of 5047
processing polygon 24 of 5047
processing polygon 25 of 5047
processing polygon 26 of 5047
processing polygon 27 of 5047
processing polygon 28 of 5047
processing polygon 29 of 5047
processing polygon 30 of 5047
processing polygon 31 of 5047
processing polygon 32 of 5047
processing polygon

Traceback (most recent call last):
  File "C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\_ffi\loop.py", line 269, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
KeyboardInterrupt
2020-12-08T22:30:53Z


KeyboardInterrupt: 

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [7]:
## testing a one-hour wait

offset_dict = {
    30: 'two years'
}

exports, exceptions = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict, date_range_list
)

5047 features have been loaded
processing polygon 1 of 5047
processing polygon 2 of 5047
processing polygon 3 of 5047
processing polygon 4 of 5047
processing polygon 5 of 5047
processing polygon 6 of 5047
processing polygon 7 of 5047
processing polygon 8 of 5047
processing polygon 9 of 5047
processing polygon 10 of 5047
processing polygon 11 of 5047
processing polygon 12 of 5047
processing polygon 13 of 5047
processing polygon 14 of 5047
processing polygon 15 of 5047
processing polygon 16 of 5047
processing polygon 17 of 5047
processing polygon 18 of 5047
processing polygon 19 of 5047
processing polygon 20 of 5047
processing polygon 21 of 5047
processing polygon 22 of 5047
processing polygon 23 of 5047
processing polygon 24 of 5047
processing polygon 25 of 5047
processing polygon 26 of 5047
processing polygon 27 of 5047
processing polygon 28 of 5047
processing polygon 29 of 5047
processing polygon 30 of 5047
processing polygon 31 of 5047
processing polygon 32 of 5047
processing polygon

Traceback (most recent call last):
  File "C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\_ffi\loop.py", line 269, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
KeyboardInterrupt
2020-12-09T15:33:10Z


KeyboardInterrupt: 

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [3]:
## Misha polygon pull

config_dict = load_config('config.yml')
source = config_dict['data_list'][0]
sensor = config_dict['sensors'][1]
# export_folder = config_dict['drive_folder']
export_folder = config_dict['bucket']

print(export_folder)
print(source)
print(sensor)

project-canopy-temp-2
{'name': ['dynamic_date_range_v4_secondary_sort_using_area_full_band_output'], 'start_date': '2019-01-01', 'end_date': '2020-12-31', 'geometry': 'point', 'size': 20000, 'resolution': 10, 'sort_by': 'name', 'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI', 'sensors': [0, 1, 2]}
{'name': 'COPERNICUS/S2_SR', 'prefix': 'S2_CloudFree', 'type': 'opt', 'bands': ['B2', 'B3', 'B4', 'B8', 'B8A', 'TCI_R', 'TCI_G', 'TCI_B', 'NDVI'], 'filters_after': [{'CLOUDY_PERCENTAGE': {'lte': 10}}]}


C:\Users\David\canopy\canopy-gis\google-earth-explore\GEE_DataDownloader\download_sen12.py:442: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


In [4]:
fc,date_range_list,df_labels = import_aois(david__label_path)

In [7]:
offset_dict = {
    45: 90,
    90: 180,
    180: 'two years'
}

exports, exceptions = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict, date_range_list, loop_start=81
)

101 features have been loaded
processing polygon 82 of 101
processing polygon 83 of 101
processing polygon 84 of 101
processing polygon 85 of 101
processing polygon 86 of 101
processing polygon 87 of 101
processing polygon 88 of 101
processing polygon 89 of 101
processing polygon 90 of 101
processing polygon 91 of 101
processing polygon 92 of 101
processing polygon 93 of 101
processing polygon 94 of 101
processing polygon 95 of 101
processing polygon 96 of 101
processing polygon 97 of 101
processing polygon 98 of 101
processing polygon 99 of 101
processing polygon 100 of 101
processing polygon 101 of 101


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
